In [1]:
import sys
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1681141723257_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName('stock-summary').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
databucket = "databucket-us-east-1-407925949"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.csv("s3://" + databucket + "/data/stock_prices.csv", header=True, inferSchema=True).select("Trade_Date", "Ticker", "Close", "Volume")
df.sort(df.Trade_Date, ascending=True).show(7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------------------+----------+
|Trade_Date|Ticker|             Close|    Volume|
+----------+------+------------------+----------+
|2020-01-02|    sq| 63.83000183105469| 5264700.0|
|2020-01-02|  aapl|  75.0875015258789|1.354804E8|
|2020-01-02|  amzn| 1898.010009765625| 4029000.0|
|2020-01-02|     m|16.520000457763672| 2.63881E7|
|2020-01-02|  tsla|   86.052001953125| 4.76605E7|
|2020-01-02|  msft| 160.6199951171875| 2.26221E7|
|2020-01-02|    ge|11.930000305175781| 8.74218E7|
+----------+------+------------------+----------+
only showing top 7 rows

In [6]:
("Total number of stocks records: " + str(df.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Total number of stocks records: 1771'

In [7]:
dfVol = df.filter(df.Volume > 10000000).sort(df.Volume, ascending=False)
dfVol.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------------------+----------+
|Trade_Date|Ticker|             Close|    Volume|
+----------+------+------------------+----------+
|2020-02-28|  aapl| 68.33999633789062|  4.2651E8|
|2020-03-12|  aapl|62.057498931884766| 4.18474E8|
|2020-03-20|  aapl|57.310001373291016|4.016932E8|
|2020-07-31|  aapl|106.26000213623047|3.743368E8|
|2020-03-13|  aapl| 69.49250030517578| 3.70732E8|
|2020-08-24|  aapl|125.85749816894531|3.459376E8|
|2020-03-02|  aapl| 74.70249938964844|3.413972E8|
|2020-08-21|  aapl|124.37000274658203|3.380548E8|
|2020-03-23|  aapl|56.092498779296875|3.367528E8|
|2020-09-04|  aapl|120.95999908447266|3.326072E8|
+----------+------+------------------+----------+
only showing top 10 rows

In [8]:
df.createOrReplaceTempView("stockprice")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dfSql = spark.sql("SELECT Trade_Date, Ticker, round(DOUBLE(Close),2) AS Closing_Value, Volume  FROM stockprice WHERE Volume > 10000000 ORDER BY Close DESC LIMIT 10")
dfSql.sort(dfSql.Volume, ascending=False).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------+----------+
|Trade_Date|Ticker|Closing_Value|    Volume|
+----------+------+-------------+----------+
|2020-12-18|  tsla|        695.0|2.221262E8|
|2020-12-31|  tsla|       705.67| 4.96499E7|
|2020-12-30|  tsla|       694.78|  4.2846E7|
|2020-12-28|  tsla|       663.69| 3.22786E7|
|2020-12-29|  tsla|       665.99| 2.29108E7|
|2020-01-31|  amzn|      2008.72| 1.55673E7|
|2020-04-16|  amzn|      2408.19| 1.20382E7|
|2020-03-12|  amzn|      1676.61| 1.13462E7|
|2020-03-17|  amzn|      1807.84| 1.09171E7|
|2020-03-19|  amzn|      1880.93| 1.03999E7|
+----------+------+-------------+----------+

Movie Analysis

In [10]:
spark = SparkSession.builder.appName("movie-summary").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_challenge = spark.read.csv("s3://challengebucket-us-east-1-407925949/data/movies.csv", header=True, inferSchema=True).select('year', 'title', 'directors_0', 'rating', 'actors_0', 'actors_1', 'actors_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
dfJodieFoster = df_challenge.filter((df_challenge.actors_0 == "Jodie Foster" ) | (df_challenge.actors_1 == "Jodie Foster") | (df_challenge.actors_2 == "Jodie Foster")).sort(df_challenge.year, ascending=True).show() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+------------------+------+-----------------+-------------------+------------------+
|year|               title|       directors_0|rating|         actors_0|           actors_1|          actors_2|
+----+--------------------+------------------+------+-----------------+-------------------+------------------+
|1976|The Little Girl W...|   Nicolas Gessner|   7.0|     Jodie Foster|       Martin Sheen|      Alexis Smith|
|1976|        Bugsy Malone|       Alan Parker|   6.5|     Jodie Foster|         Scott Baio|    Florrie Dugger|
|1991|The Silence of th...|    Jonathan Demme|   8.7|     Jodie Foster|    Anthony Hopkins|Lawrence A. Bonney|
|1994|                Nell|     Michael Apted|   6.3|     Jodie Foster|        Liam Neeson|Natasha Richardson|
|1994|            Maverick|    Richard Donner|   6.9|       Mel Gibson|       Jodie Foster|      James Garner|
|1997|             Contact|   Robert Zemeckis|   7.3|     Jodie Foster|Matthew McConaughey|      Tom Skerritt|
|